Before starting let's go back to out main directory. In USERNAME write the user path you are using...

In [ ]:
import os 
os.chdir('C:/Users/USERNAME/')

During this phase, we will do two things:

->Get familiar with the functions used for motion capture.

->Reorient the global coordinate system of the cameras to make it much easier to later construct the bounding boxes used to detect the component being worked on.

Before doing anything, record some videos (always under 1 minute) with the following characteristics:

->Hand movements should not be too fast.

->The videos should start with the operator slowly opening and closing both hands above the component.

However, there is still one important consideration: in order to simplify the later description of volumes (bounding boxes of components) in 3D space, the global coordinate system must at least be consistent and aligned with certain key points (belonging to the unit on the workbench) chosen arbitrarily by the user.

We will therefore choose to record videos in which the user, moving their right index finger (marked by keypoint number 8), points to 3 locations on the component:

-The origin of the new coordinate system (referred to as O from now on)

-A point along the new X-axis  (referred to as X from now on)

-A point along the new Y-axis (referred to as Y from now on)

Important note! The user must ensure that these points are chosen so that there is already an orthogonal geometry between the X and Y points (for example, if thinking about a motherboard, it would be advisable to place these two points on two opposite corners).
Important note 2! To avoid "non-perfectly aligned global coordinate systems" due to small deviations between the real 3D position of the keypoint and the triangulated position calculated by the motion capture functions, it is recommended to place the X and Y points as far as possible from the chosen origin, while still remaining on the component

To begin, using the commands below, we will create a folder named videos inside EasyMocap/data and upload the calibration files into it.

Next, record and upload the videos you want to use for motion capture to fine-tune the reference system.
Regarding the video specifications, the same considerations used during calibration apply.

After uploading the videos, also load the extri.yml and intri.yml files previously obtained during the calibration phase and contained in extri_data.

In [ ]:
import os 
import shutil

os.makedirs('EasyMocap/data/videos', exist_ok=True)
shutil.copy('EasyMocap/yamls/intri.yml' ,'EasyMocap/data')
shutil.copy('EasyMocap/yamls/extri.yml' ,'EasyMocap/data')

If you end up making a mess, there’s always the cleanhandmocap.py function, which lets you clean up the folder you’re working in.


In [ ]:
#!python EasyMocap/apps/hopeitworks/cleanhandmocap.py

The next function, to be used only once (the file will remain modified afterward), allows you to adjust certain parameters depending on whether the person in the video is right-handed or left-handed. Its use is recommended.

YOU HAVE TO RUN IT IN THE TERMINAL

run the command "  cd ..  " multiple times untill you arrive at the point where the terminal is like: PS C:\Users\USERNAME>

then run:

python EasyMocap/apps/hopeitworks/choosehand.py

and follow the instructions

Ok. Now let's move on to extracting the images using the next notebook... make sure you're always in the main directory, using the command at the beginning of the notebook.

In [ ]:
##ESTRAE IMMAGINI DA VIDEO
import time
import os
# Inizio del timer
start_time = time.time()

# ESTRAZIONE IMMAGINI
!python EasyMocap/apps/preprocess/extract_image.py EasyMocap/data

# Fine del timer
end_time = time.time()
elapsed_time = end_time - start_time
# Salva il tempo in un file txt
with open('EasyMocap/data/times.txt', 'a') as f:
    f.write(f't extract_image, {elapsed_time:.2f}\n')

Keypoints computation...

In [ ]:
##ESTRAE KEYPOINTS
import time
import os
import json
import shutil
# Inizio del timer
start_time = time.time()

## MANO SINISTRA
!python EasyMocap/apps/preprocess/extract_keypoints.py EasyMocap/data --mode mp-handl
## RINOMINA ANNOTS IN ANNOTSL PER CONVENIENZA DEL SUCCESSIVO ANNOT (DELLA MANO DESTRA)
os.rename('EasyMocap/data/annots', 'EasyMocap/data/annotsl')
## MANO DESTRA
!python EasyMocap/apps/preprocess/extract_keypoints.py EasyMocap/data --mode mp-handr


##INIZIO JOIN DELLE DUE CARTELLE ANNOTS

# Define the path to your main folder
main_folder_path = 'EasyMocap/data/annotsl'

# Get a list of sub-folders within the main folder
subfolders = [f.path for f in os.scandir(main_folder_path)]

# Loop through each sub-folder
for subfolder_path in subfolders:

    # Get a list of JSON files within the sub-folder
    json_files = [f.path for f in os.scandir(subfolder_path)]

    # Loop through the JSON files in the sub-folder
    for json_file_path in json_files:

        # You can now read and process the JSON file
        with open(json_file_path, 'r') as json_file:
            data = json.load(json_file)
            json_file_path_dest = json_file_path.replace("annotsl", "annots")

            # Step 1: Read the existing data from the destination JSON file
            with open(json_file_path_dest, 'r') as json_file:
                existing_data = json.load(json_file)

            # Step 2: Modify or add new data to the existing data
            existing_data["annots"][0]["handl2d"] =  data["annots"][0]["handl2d"]
            existing_data["annots"][0]["bbox_handl2d"] =  data["annots"][0]["bbox_handl2d"]

            # Step 3: Write the updated data back to the JSON file
            with open(json_file_path_dest, 'w') as json_file:
                json.dump(existing_data, json_file, indent=4)

shutil.rmtree('EasyMocap/data/annotsl')
##FINE JOIN DELLE DUE CARTELLE ANNOTS

# Fine del timer
end_time = time.time()
elapsed_time = end_time - start_time
# Salva il tempo in un file txt
with open('EasyMocap/data/times.txt', 'a') as f:
    f.write(f't extract_keypoints, {elapsed_time:.2f}\n')

##in times.txt, SCRIVERE A INIZIO FILE NUMERO FRAME E NUMERO FPS CON QUESTO FORMATO EX. (#FRAMES, 434 /n #FPS, 9)

Finally triangulation...

In [ ]:
##TRIANGOLAZIONE K2D->K3D
import time

# Inizio del timer
start_time = time.time()

!python EasyMocap/apps/fit/triangulate1p.py --cfg_data EasyMocap/config/recon/mv1p.yml --opt_data args.path EasyMocap/data args.out EasyMocap/data/output-keypoints3d --cfg_exp EasyMocap/config/recon/triangulator-mv1p-total.yml --opt_exp  args.debug True

# Fine del timer
end_time = time.time()
elapsed_time = end_time - start_time
# Salva il tempo in un file txt
with open('EasyMocap/data/times.txt', 'a') as f:
    f.write(f't triangulate, {elapsed_time:.2f}\n')

If everything went well, we can proceed by navigating to the folder `C:\Users\USERNAME\EasyMocap\data\output-keypoints3d\match` (using Windows Folders explorer is faster).  

We can then scroll through the images by holding down the arrow key (the images are located in `C:\Users\USERNAME\EasyMocap\data\output-keypoints3d\match`) and select the frames corresponding to the **O**, **X**, and **Y** reference points—i.e., where the fingertip is best aligned with the three points we discussed earlier.  

Now, note down the frame numbers for these three keyframes, then go to the folder `EasyMocap/data/output-keypoints3d/keypoints3d`, open the corresponding JSON files for the selected frames, and extract the three coordinate values for `handr 8` (right hand index finger).  

Write these values into a file named `newcoordinates.txt` (you can find it in `EasyMocap/data` with some placeholder values—just overwrite them) using the following format:  

O: 0.1656968, 0.4882543, 0.2202636

X: 0.0809912, 0.4186054, 0.2154134

Y: 0.2596883, 0.3956782, 0.2286461

**Important:** Do **not** copy indices `0`, `1`, and `2`—they represent the coordinate axes (X, Y, Z), not the actual values! And obviously, remember to save the updated `newcoordinates.txt`.  

Now we begin the REORIENTATION phase

FIRST STEP
We transform the extrinsic calibration file so that it describes the rototranslations needed to convert from the GLOBAL reference system to the LOCAL reference system.

This is necessary because the calibration process outputs an extrinsic file describing the inverse transformation—that is, the rototranslation required to go from the LOCAL reference system (each camera's frame) to the GLOBAL reference system (in this case, fixed at the origin of the checkerboard used during calibration).

By modifying this file, we ensure it correctly represents the desired world-to-camera transformation instead of the original camera-to-world relationship.

In [ ]:
!python EasyMocap/apps/hopeitworks/rototrasla_yaml.py EasyMocap/data/extri.yml

SECOND STEP
To confirm what has been done so far, we can use the next function to plot the .yml file obtained from the previous function. The local reference systems now take on a tangible three-dimensional meaning, traceable back to the real configuration of the system.

In [ ]:
#!python EasyMocap/apps/hopeitworks/visualizza_yaml.py EasyMocap/data/extri_inv.yml

THIRD STEP
Now we can proceed with the actual reorientation of the global reference system using the following function (which utilizes the newcoordinates text file written earlier).

In [ ]:
!python EasyMocap/apps/hopeitworks/riorienta_sistemaglobale.py EasyMocap/data/extri_inv.yml EasyMocap/data/newcordinate.txt

FOURTH STEP
By invoking the 3D visualization function again, we can check whether the global system has been correctly reoriented.

In [ ]:
#!python EasyMocap/apps/hopeitworks/visualizza_yaml.py EasyMocap/data/extri_inv_reoriented.yml

FIFTH STEP
It is essential to return to the original point of view (POV) used initially.
This is done by rototranslating the file again to return to a local-to-global representation.

In [ ]:
!python EasyMocap/apps/hopeitworks/rototrasla_yaml.py EasyMocap/data/extri_inv_reoriented.yml

Now let’s make the new extri file the official one by running the replacement function:
sostituisciextri.py

(This script will overwrite/update the existing extrinsic calibration file with the newly adjusted parameters.)

In [ ]:
!python EasyMocap/apps/hopeitworks/sostituisciextri.py EasyMocap/data

Now let’s verify if our new reference system is correct and properly aligned with our component using the cube function (the same one used during calibration).

You’ll find the results in the folder:
EasyMocap/data/cube

In [ ]:
!python EasyMocap/apps/calibration/check_calib.py EasyMocap/data --out EasyMocap/data --mode cube --write

Most of the time, some adjustments will be needed... Don’t worry, it’s very simple. Our reference system might not be centered at the origin, or the X and Y axes might not be aligned with our component. To fix this, just repeat the previous steps iteratively until the desired result is achieved.

To do this, change the values related to a single variable (either O, X, or Y) in the newtuning.txt file one at a time (to avoid drastic changes). Remember, the values are in meters.

For example:

If you want the new X-axis to rotate clockwise around the Z-axis, enter a negative Y value.

Also, keep in mind that the reorient function tries to make the axes of the "new" global system pass through the coordinates of the points specified in the "previous" global system (for clarity).

Once you’ve updated newtuning.txt, proceed with the cell below.

In [ ]:
#ITERAZIONE ripetiamo comandi precedenti e controlliamo proiezione cubo...
!python EasyMocap/apps/hopeitworks/rototrasla_yaml.py EasyMocap/data/extri.yml
!python EasyMocap/apps/hopeitworks/riorienta_sistemaglobale.py EasyMocap/data/extri_inv.yml EasyMocap/data/newtuning.txt
!python EasyMocap/apps/hopeitworks/rototrasla_yaml.py EasyMocap/data/extri_inv_reoriented.yml
!python EasyMocap/apps/hopeitworks/sostituisciextri.py EasyMocap/data
!python EasyMocap/apps/hopeitworks/ripristinatuning.py EasyMocap/data
!python EasyMocap/apps/calibration/check_calib.py EasyMocap/data --out EasyMocap/data --mode cube --write

OPTION A: Further tuning required → Write the new desired values in newtuning.txt, then run the cell #ITERAZIONE again.

OPTION B: We are happy with the results.
In this case, we can save the results in YAML files using the cell below. Our task is complete! :)

In [ ]:
import shutil

shutil.copy('EasyMocap/data/extri.yml', 'EasyMocap/yamls/REORIENTED')
shutil.copy('EasyMocap/data/intri.yml', 'EasyMocap/yamls/REORIENTED')

See you in the next tutorial